pip install catboost

pip install xgboost

pip install lightgbm

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import itertools
import plotly.graph_objects as go

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier, StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from catboost import CatBoostRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


import warnings
warnings.simplefilter(action="ignore")

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
test_data = pd.read_csv('test.csv')
print(test_data.shape)
test_data.head(2)

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.500,0,0,330911,7.829,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.000,1,0,363272,7.000,NaN,S


In [4]:
train_data = pd.read_csv('train.csv')
print(train_data.shape)
train_data.head(2)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C


In [5]:
gen_sub = pd.read_csv("gender_submission.csv")
print(gen_sub.shape)
gen_sub.head(2)

(418, 2)


,PassengerId,Survived
0,892,0
1,893,1


In [6]:
sub_id=test_data["PassengerId"]
sub_id.head(1)

0    892
Name: PassengerId, dtype: int64

In [7]:
first_train = train_data.shape[0]
first_train

891

In [8]:
df = pd.concat([train_data, test_data]).reset_index()

In [9]:
df.head(2)

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0.000,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,1,2,1.000,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C


# EDA

In [10]:
def check_df(dataframe, head=5):
    print('##################### Shape #####################')
    print(dataframe.shape)
    print('##################### Types #####################')
    print(dataframe.dtypes)
    print('##################### Head #####################')
    display(dataframe.head(head))
    print('##################### Tail #####################')
    display(dataframe.tail(head))
    print('##################### NA #####################')
    print(dataframe.isnull().sum())
    print('##################### Quantiles #####################')
    display(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [11]:
check_df(df)

##################### Shape #####################
(1309, 13)
##################### Types #####################
index            int64
PassengerId      int64
Survived       float64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
##################### Head #####################


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0.000,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,1,2,1.000,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,2,3,1.000,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,3,4,1.000,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,4,5,0.000,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


##################### Tail #####################


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1304,413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.050,NaN,S
1305,414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000,0,0,PC 17758,108.900,C105,C
1306,415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500,0,0,SOTON/O.Q. 3101262,7.250,NaN,S
1307,416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.050,NaN,S
1308,417,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.358,NaN,C


##################### NA #####################
index             0
PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64
##################### Quantiles #####################


,count,mean,std,min,0%,5%,50%,95%,99%,100%,max
index,1309.000,369.479,248.767,0.000,0.000,32.400,327.000,824.600,876.920,890.000,890.000
PassengerId,1309.000,655.000,378.020,1.000,1.000,66.400,655.000,1243.600,1295.920,1309.000,1309.000
Survived,891.000,0.384,0.487,0.000,0.000,0.000,0.000,1.000,1.000,1.000,1.000
Pclass,1309.000,2.295,0.838,1.000,1.000,1.000,3.000,3.000,3.000,3.000,3.000
Age,1046.000,29.881,14.413,0.170,0.170,5.000,28.000,57.000,65.000,80.000,80.000
SibSp,1309.000,0.499,1.042,0.000,0.000,0.000,0.000,2.000,5.000,8.000,8.000
Parch,1309.000,0.385,0.866,0.000,0.000,0.000,0.000,2.000,4.000,9.000,9.000
Fare,1308.000,33.295,51.759,0.000,0.000,7.225,14.454,133.650,262.375,512.329,512.329


In [13]:
drop_list = ["Name","Ticket","Cabin","PassengerId","index"]

In [14]:
df.drop(drop_list, axis=1, inplace=True)

In [15]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.000,3,male,22.000,1,0,7.250,S
1,1.000,1,female,38.000,1,0,71.283,C


In [16]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Returns the names of categorical, numeric and categorical but cardinal variables in the data set.
    Note Categorical variables include categorical variables with numeric appearance.

    Parameters
    ------
        dataframe: d/ataframe
                Variable names of the dataframe to be taken
        cat_th: int, optional
                class threshold for numeric but categorical variables
        car_th: int, optinal
                class threshold for categorical but cardinal variables

    Returns
    ------
        cat_cols: list
                Categorical variable list
        num_cols: list
                Numeric variable list
        cat_but_car: list
                List of cardinal variables with categorical appearance

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = total number of variables
        num_but_cat is inside cat_cols.
        The sum of the 3 return lists equals the total number of variables: cat_cols + num_cols + cat_but_car = number of variables

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"] 

    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]

    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]

    cat_cols = cat_cols + num_but_cat

    cat_cols = [col for col in cat_cols if col not in cat_but_car] 

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"] 

    num_cols = [col for col in num_cols if col not in num_but_cat] 
    
    print(f"Observations: {dataframe.shape[0]}") 
    print(f"Variables: {dataframe.shape[1]}") 
    print(f'cat_cols: {len(cat_cols)}') 
    print(f'num_cols: {len(num_cols)}') 
    print(f'cat_but_car: {len(cat_but_car)}') 
    print(f'num_but_cat: {len(num_but_cat)}') 


    return cat_cols, num_cols, cat_but_car, num_but_cat

In [17]:
cat_cols, num_cols, cat_but_car,  num_but_cat = grab_col_names(df)

Observations: 1309
Variables: 8
cat_cols: 6
num_cols: 2
cat_but_car: 0
num_but_cat: 4


In [19]:
cat_cols

['Sex', 'Embarked', 'Survived', 'Pclass', 'SibSp', 'Parch']

In [20]:
cat_cols.remove("Survived")

In [21]:
cat_cols

['Sex', 'Embarked', 'Pclass', 'SibSp', 'Parch']

In [22]:
num_cols

['Age', 'Fare']

In [23]:
cat_but_car

[]

In [24]:
num_but_cat

['Survived', 'Pclass', 'SibSp', 'Parch']